In [21]:
! pip install openai==0.28

In [1]:
import pandas as pd
import openai
from time import strftime

/var/folders/m_/znmxvld547z2ynkh41wpdqm40000gn/T/ipykernel_1289/1655272596.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def prompt(Q):
  tries = 0
  while tries<5:
    try:
      response = openai.ChatCompletion.create(
        model=mmodel,
        messages=Q,
        temperature=0.5,
        #max_tokens=tkn,
        frequency_penalty=0,
        presence_penalty=0,
        request_timeout=199
      )['choices'][0]['message']['content']
      break
    except Exception as e:
      tries = tries+1
      print(f"An error occurred {tries:d}th time: {e}")
  return(Q,response)

In [3]:
dtime = strftime("%Y_%m_%d-%H%M%S")

In [4]:
mmodel = "gpt-3.5-turbo-1106"
openai.api_key = "YOUR_API_KEY"

In [5]:
PROPERTY = 'Yield strength'

system = 'You follow instructions carefully, you provide as many rows of the table as possible. Before giving me the answer'

qs = ["Provide me with a list of "+PROPERTY+" values for different high entropy alloys materials. Your response should be a table consisting of 2 columns: material, value. The materials have to be typed as unique chemical compositions consisting of chemical element abbreviations and numbers only (e.g. GaAs, but not Gallium Arsenide). The values have to be single numbers, not ranges. Type out as many different values as you can. You are not allowed to type anything else than this table.",
      "Continue expanding this table with new values of "+PROPERTY+" making sure you do not duplicate entries. Type out as many different values as you can. You are not allowed to type anything else than this table."]


In [6]:
filename = PROPERTY.replace(" ", "")+'_'+dtime+'.csv'
print(f"Saving to: {filename} and {filename.replace('csv','txt')}")

Saving to: Yieldstrength_2024_04_09-163633.csv and Yieldstrength_2024_04_09-163633.txt


In [7]:
sss = [{"role": "system", "content": system}]
tab = []
tab_clean = []
ur = 0
um = 0
i = 0

In [8]:
while True:
  # here i send my first prompt first and then loop over the second one over and
  # over again. You will likely have a different approach to this.
  if i<1:
    qq = qs[0]
  else:
    qq = qs[1]
  # save the first prompt to the conversation
  sss.append({"role": "user", "content": qq})
  # send out the first prompt and receive the response
  sss,ans = prompt(sss)
  # save the response to the conversation
  sss.append({"role": "assistant", "content": ans})

  # we are saving the raw prompts and raw responses, in case we want to analyze
  # or postprocess later
  with open(filename.replace('csv','txt'), 'a') as file:
    print("USER: "+qq, file=file)
    print("GPT : "+ans, file=file)

  # Here we start to grab the data into a nicer structure. For simplicity I had
  # a lot of assumptions. I assume that the word 'value' exists in the header
  # (see first prompt), I remove the header, the separator, and get the rest.
  lines = ans.split('\n')
  if 'value' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')
  if '----' in lines[0].lower():
    ans = '\n'.join(lines[1:])
  lines = ans.split('\n')

  # here I try to split the table by columns. I'm assuming that they are
  # separated with |, which is not necessary always the case.
  tab.append(ans)
  try:
    for line in tab[-1].strip().split('\n'):
      tab_clean.append(line.strip('|').split('|'))
  except:
    pass

  # another assumption - only two columns, "material" and "value" (see prompt)
  # some cleanup, converting strings to numbers etc.
  # there is no error handling or edge cases, for example 1.6-1.7 will be removed
  # because it technically is not a number (although it kind of is).
  df = pd.DataFrame(tab_clean, columns=['Material', 'Value'])
  df = df[pd.to_numeric(df['Value'], errors='coerce').notna()]
  df['Value'] = pd.to_numeric(df['Value'])
  df.to_csv(filename, index=False)

  # here I count how many new (non-duplicate) materials we are extracting each
  # time, to monitor progress. We stop if more than 10 iterations or not progress
  if len(df.drop_duplicates()) > ur or df['Material'].nunique() > um:
    ur = len(df.drop_duplicates())
    um = df['Material'].nunique()
    i = i+1
    if i > 10:
      print("Stopping due to 10 iterations exceeded")
      break
  else:
    print("Stopping due to NO PROGRESS")
    break

  print(f"Iteration: {i:3} Generated_rows: {len(lines):3};     TOTAL:  Uniq_rows: {len(df.drop_duplicates()):4d}   Uniq_materials: {df['Material'].nunique():4d}")

print(df)

Iteration:   1 Generated_rows:  14;     TOTAL:  Uniq_rows:   10   Uniq_materials:   10
Iteration:   2 Generated_rows:  15;     TOTAL:  Uniq_rows:   15   Uniq_materials:   15
Iteration:   3 Generated_rows:  19;     TOTAL:  Uniq_rows:   19   Uniq_materials:   19
Stopping due to NO PROGRESS
          Material  Value
4      CoCrFeMnNi     655
5      AlCrCuFeNi     580
6      FeCoNiCrMn     650
7      AlCoCrFeNi     570
8    CoCrFeMnNiTi     720
9     CoCrFeMnNiV     700
10   CoCrFeMnNiMo     750
11     AlCuCrFeNi     530
12     CoCuFeMnNi     610
13       CoCrFeNi     620
14     CoCrFeMnNi     655
15     AlCrCuFeNi     580
16     FeCoNiCrMn     650
17     AlCoCrFeNi     570
18   CoCrFeMnNiTi     720
19    CoCrFeMnNiV     700
20   CoCrFeMnNiMo     750
21     AlCuCrFeNi     530
22     CoCuFeMnNi     610
23       CoCrFeNi     620
24      FeMoNbTaW     790
25     CoCrFeNiMn     680
26     CoCrFeNiPd     730
27     AlCoCrFeMn     540
28     CoCrFeNiSi     640
29     CoCrFeMnNi     655
30     Al

In [9]:
# load GTD
gtd_df = pd.read_csv('yield_strength_GTD.csv')
gtd_df

,material,value,passage
0,Alloy 82/182,420.0,Chapter 12 Engineering Applications. Overall t...
1,CoNiFe(AlSi)0.2,342.0,2 High-entropy alloys: basic concepts. In the ...
2,HEA,1262.0,6 Synthesis and processing. The SPSed alloy ha...
3,SWCed,630.0,6 Synthesis and processing. The relative densi...
4,HEA,315.0,6 Synthesis and processing. The FSP of HEA ref...
...,...,...,...
8789,CoCrFeMnNi (0 at.%% Mo),216.0,Improved tribological performance of nickel ba...
8790,5 wt% TiC-reinforced HEA,698.0,Effect of ZrO2 particles on the nanomechanical...
8791,Al0.3CoCrFeMnNi-3 vol% Y2O3,1760.0,Effect of ZrO2 particles on the nanomechanical...
8792,Al0.3CoCrFeMnNi-0 vol% Y2O3,980.0,Effect of ZrO2 particles on the nanomechanical...


In [16]:
check_list = list(df['Material'])
gen_value = list(df['Value'])
gtd_list = list(gtd_df['material'])
gtd_value = list(gtd_df['value'])

err = []
for m in check_list:
    if m.strip() in gtd_list:
        val = gen_value[check_list.index(m)]
        cor = gtd_value[gtd_list.index(m.strip())]
        err.append(abs(val - cor) / cor)
    else:
        material = m.strip()
        found = False
        for gtd in gtd_list:
            if material in gtd:
                prev_idx = gtd.index(material) - 1
                next_idx = gtd.index(material) + len(material)
                if prev_idx > 0 and (gtd[prev_idx].isalpha() or gtd[prev_idx].isdigit()):
                    continue
                if next_idx < len(gtd) and (gtd[next_idx].isalpha() or gtd[next_idx].isdigit()):
                    continue 
                val = gen_value[check_list.index(m)]
                cor = gtd_value[gtd_list.index(gtd)]
                err.append(abs(val - cor) / cor)
                found = True
                break
        if not found:
            err.append(-1)

print("Total:", len(check_list))
print("Found:", len(err) - err.count(-1))
print("Unkdown:", err.count(-1))
for i in range(len(err)):
    if err[i] == -1:
        err[i] = None
filtered_list = list(filter(None, err))
print("average error:", sum(filtered_list) / len(filtered_list))
result_df = pd.DataFrame({
    "Material": check_list,
    "precentage of error": err
})
result_df

Total: 44
Found: 23
Unkdown: 21
average error: 0.5620184819104491


,Material,precentage of error
0,CoCrFeMnNi,0.212963
1,AlCrCuFeNi,NaN
2,FeCoNiCrMn,1.096774
3,AlCoCrFeNi,0.544000
4,CoCrFeMnNiTi,NaN
5,CoCrFeMnNiV,0.578313
6,CoCrFeMnNiMo,NaN
7,AlCuCrFeNi,NaN
8,CoCuFeMnNi,0.322222
9,CoCrFeNi,0.483333
